In [12]:
import pandas as pd

df = pd.read_csv('missionData.csv', encoding='latin-1')

print(df.head())

                                            positive  \
0  ##Training Scenario: Operation Silent Thunder\...   
1  ##Training Scenario: Operation Iron Wave\n**Co...   
2  ##Training Scenario: Operation Thunderstrike\n...   
3  ##Training Scenario: Operation Desert Storm\n*...   
4  ##Training Scenario: Operation Steel Talon\n**...   

                                            negative  
0  ##Training Scenario: Operation Silent Thunder\...  
1  ##Training Scenario: Operation Iron Wave\n**Co...  
2  ##Training Scenario: Operation Thunderstrike\n...  
3  ##Training Scenario: Operation Desert Storm\n*...  
4  ##Training Scenario: Operation Steel Talon\n**...  


In [13]:
data_list = df.values.tolist()
new_data_list = [] 

# Insert a space at the third index of each row
for i in range(50):
    new_row = []
    for entry in data_list[i]:  
        new_entry = entry[0:2] + ' ' + entry[2:]
        new_entry = new_entry.replace('**\n', '**').replace('**', '\n**').replace(':\n**', ':**').replace('*', '-').replace('--', '**').replace('', "'")
        new_entry = new_entry[:100] + new_entry[100:].replace('\n\n**', '\n**')
        new_row.append(new_entry)
    new_data_list.append(new_row)

new_column_names = list(df.columns)
new_df = pd.DataFrame(new_data_list, columns=new_column_names)

print(new_df.head())

                                            positive  \
0  ## Training Scenario: Operation Silent Thunder...   
1  ## Training Scenario: Operation Iron Wave\n\n*...   
2  ## Training Scenario: Operation Thunderstrike\...   
3  ## Training Scenario: Operation Desert Storm\n...   
4  ## Training Scenario: Operation Steel Talon\n\...   

                                            negative  
0  ## Training Scenario: Operation Silent Thunder...  
1  ## Training Scenario: Operation Iron Wave\n\n*...  
2  ## Training Scenario: Operation Thunderstrike\...  
3  ## Training Scenario: Operation Desert Storm\n...  
4  ## Training Scenario: Operation Steel Talon\n\...  


In [14]:
print(new_df.iloc[0]['positive'])

## Training Scenario: Operation Silent Thunder

**Commander's Intent:**
- Destroy the REDLAND's radar installation and supply depot.
- Ensure 90% bombs on target.
- Maximum 40% BLUELAND Air Force losses.
- Minimize REDLAND's ability to detect incoming aircraft.

**Force Composition:**
- REDLAND Air Force: three S-300 surface-to-air missile systems, six MiG-29 fighter jets, and four ZSU-23-4 Shilka self-propelled anti-aircraft gun systems.
- REDLAND Ground Forces: 800 troops armed with light infantry weapons and RPGs.

**Geography:**
- Distance to REDLAND threats: Radar installation: 5 km west of the target, Supply depot: 8 km north of the target.
- Environmental Conditions: Dawn mission with clear skies and visibility of 10 km. Target located in a hilly area with sparse vegetation.

**Surrounding Conditions:**
- REDLAND's Air Force is conducting routine patrols, with increased radar surveillance.
- REDLAND's ground forces have set up temporary checkpoints 10 km from the target, but no 

In [15]:
prompt_text = """
BLUELAND is a make-believe country. REDLAND is another make-believe country, an enemy of BLUELAND. 

Write me a training scenario for an air strike mission by BLUELAND to destroy a specified target. This scenario will be given to BLUELAND, who will devise a strategy to execute the plan. The scenario should be non-descriptive, concise and factual. The information should be sufficient to help BLUELAND to decide their loadout and strategy. The scenario should include the following information about the target and REDLAND: 
1.	Commander’s Intent: Captures the broad mission objectives (eg: 100% TOT, 50% BLUELAND losses, 100% REDLAND destruction, destroy bunkers, etc)
2.	Force Composition: REDLAND’s units and objects participating in the scenario, Order of Battle, i.e the identification, strength, command structure, disposition for the personnel, units, equipment of any military force, command & control hierarchies, communication networks and communication channels. This should be specific, i.e exact model of aircraft or missiles (eg: F-15 fighter jets, SA-7 surface-to-air missile launchers, ZSU-23-4 Shilka self-propelled anti-aircraft gun systems, etc). 
3.	Geography: Distance to target and distance to REDLAND threats in nautical miles (nm). Specification of environmental conditions which are of interest (eg: day/night/dawn, buildings, vegetation, ditches in surroundings, etc) 
4.	Surrounding Conditions: Overview of all scenario-related information of interest, i.e surrounding military situation, REDLAND readiness state, etc.
5.	Target(s) Details:  Any relevant information about target(s), i.e type and number of targets. 
6.	Rules of Engagement: Directives that outline when and how BLUELAND can engage hostile forces.
"""
prompt_dict = [{"role": "user", "content": prompt_text}]

formatted_df = pd.DataFrame(columns=['prompt', 'completion', 'label'])
prompt_lst = []
completion_lst = [] 
label_lst = [] 

for row in new_df['positive']:
    prompt = tokenizer.apply_chat_template(prompt_dict, tokenize=False, add_generation_prompt=True)
    completion = prompt + row + "<eos>"
    prompt_lst.append(prompt)
    completion_lst.append(completion)
    label_lst.append(True) 

for row in new_df['negative']:
    prompt = tokenizer.apply_chat_template(prompt_dict, tokenize=False, add_generation_prompt=True)
    completion = prompt + row + "<eos>"
    prompt_lst.append(prompt)
    completion_lst.append(completion)
    label_lst.append(False) 

formatted_df['prompt'] = prompt_lst
formatted_df['completion'] = completion_lst
formatted_df['label'] = label_lst

print(formatted_df.head())

                                              prompt  \
0  <bos><start_of_turn>user\nBLUELAND is a make-b...   
1  <bos><start_of_turn>user\nBLUELAND is a make-b...   
2  <bos><start_of_turn>user\nBLUELAND is a make-b...   
3  <bos><start_of_turn>user\nBLUELAND is a make-b...   
4  <bos><start_of_turn>user\nBLUELAND is a make-b...   

                                          completion  label  
0  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
1  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
2  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
3  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
4  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  


In [16]:
from trl import KTOConfig, KTOTrainer, ModelConfig, get_peft_config, setup_chat_format
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-7b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-1.1-7b-it",
    device_map="cuda",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
print(formatted_df['prompt'][0])

<bos><start_of_turn>user
BLUELAND is a make-believe country. REDLAND is another make-believe country, an enemy of BLUELAND. 

Write me a training scenario for an air strike mission by BLUELAND to destroy a specified target. This scenario will be given to BLUELAND, who will devise a strategy to execute the plan. The scenario should be non-descriptive, concise and factual. The information should be sufficient to help BLUELAND to decide their loadout and strategy. The scenario should include the following information about the target and REDLAND: 
1.	Commander’s Intent: Captures the broad mission objectives (eg: 100% TOT, 50% BLUELAND losses, 100% REDLAND destruction, destroy bunkers, etc)
2.	Force Composition: REDLAND’s units and objects participating in the scenario, Order of Battle, i.e the identification, strength, command structure, disposition for the personnel, units, equipment of any military force, command & control hierarchies, communication networks and communication channels. 

In [18]:
print(formatted_df['completion'][0])

<bos><start_of_turn>user
BLUELAND is a make-believe country. REDLAND is another make-believe country, an enemy of BLUELAND. 

Write me a training scenario for an air strike mission by BLUELAND to destroy a specified target. This scenario will be given to BLUELAND, who will devise a strategy to execute the plan. The scenario should be non-descriptive, concise and factual. The information should be sufficient to help BLUELAND to decide their loadout and strategy. The scenario should include the following information about the target and REDLAND: 
1.	Commander’s Intent: Captures the broad mission objectives (eg: 100% TOT, 50% BLUELAND losses, 100% REDLAND destruction, destroy bunkers, etc)
2.	Force Composition: REDLAND’s units and objects participating in the scenario, Order of Battle, i.e the identification, strength, command structure, disposition for the personnel, units, equipment of any military force, command & control hierarchies, communication networks and communication channels. 

In [19]:
formatted_df.to_csv('missionData_llm.csv', index=False)